<a href="https://colab.research.google.com/github/flee2-trinity/Deep-Learning-in-Sports-Result-Prediction-/blob/main/Rank2019%20No%20Previous%20Season%20Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from google.colab import drive

drive.mount('/content/gdrive')
DATA_PATH = '/content/gdrive/My Drive/2021 Fall/Thesis I'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import datetime

def findFiles(path): 
  return glob.glob(path)

In [ ]:
import csv

filename = '/content/gdrive/My Drive/2021 Fall/Thesis I/NBA Data/nba 18-19/NatStat-NBA2019-Team_Statlines.csv'
with open(filename, newline='') as f:
      reader = csv.reader(f)
      """
      print(list(reader))
      ['GameDay', 'GameID', 'TeamID', 'Team', 'OpponentID', 'Opponent', 'Location',
       'Division', 'Conference', 'Playoffs', 'WinOrLoss', 'PlayerType', 'MIN', 'PTS', 
       'FGM', 'FGA', '3FM', '3FA', 'FTM', 'FTA', 'REB', 'AST', 'STL', 'BLK', 'OREB', 'TO', 'PF']
      """
      next(reader)
      rawDataL = list(reader)
print(rawDataL[0])

['2018-10-16', '985248', '5', 'Boston', '28', 'Philadelphia', 'H', '', '', '', 'W', '', '234', '103', '41', '96', '11', '37', '10', '14', '54', '21', '7', '6', '12', '14', '20']


In [ ]:
import datetime

def readTime(ampm):
  t = 0
  tL = ampm.replace(' ',':').split(':')
  if tL[2] == "PM" and int(tL[0]) < 12:
    t += 12
  t += int(tL[0])
  t += int(tL[1])/60
  return t

seasonBegin = datetime.datetime(2018, 10, 16)
noonTime = readTime("12:00 PM")
#not using
class Stats:
  def __init__(self):
    """
    MIN, PTS,FGM, FGA, 3FM, 3FA, FTM, FTA, REB, AST, STL, BLK, OREB, TO, PF
    """
    self.MIN = 0
    self.PTS = 0
    self.FGM = 0
    self.FGA = 0
    self.FM3 = 0
    self.FA3 = 0
    self.FTM = 0
    self.FTA = 0
    self.REB = 0
    self.AST = 0
    self.STL = 0
    self.BLK = 0
    self.OREB = 0
    self.TO = 0
    self.PF = 0
    self.Win = 0
    self.Loss = 0
    self.WinRate = 0
  def update(self, MIN, PTS,FGM, FGA, FM3, FA3, FTM, FTA, REB, AST, STL, BLK, OREB, TO, PF):
    self.MIN = MIN
    self.PTS = PTS
    self.FGM = FGM
    self.FGA = FGA
    self.FM3 = FM3
    self.FA3 = FA3
    self.FTM = FTM
    self.FTA = FTA
    self.REB = REB
    self.AST = AST
    self.STL = STL
    self.BLK = BLK
    self.OREB = OREB
    self.TO = TO
    self.PF = PF
  def addWinRate(self, Win, Loss):
    self.Win = Win
    self.Loss = Loss
    self.WinRate = Win/(Win+Loss)
  def notEmpty(self):
    return MIN>0

class aGame:
  def __init__(self, GameDay, GameID, TeamID, Team, OpponentID, Opponent, Location,
       Division, Conference, Playoffs, WinOrLoss, PlayerType):
    self.GameID = int(GameID)
    self.TeamID = int(TeamID)
    gdL = GameDay.split("-")
    self.GameDay = datetime.datetime(int(gdL[0]), int(gdL[1]), int(gdL[2]))
    #self.GameTime = GameTime
    self.DayCount = (self.GameDay - seasonBegin).days
    self.Team = Team
    self.OpponentID = int(OpponentID)
    self.Opponent = Opponent
    self.Home = Location == 'H'
    self.Division = Division
    self.Conference = Conference
    self.Playoffs = Playoffs == 'Y'
    self.WinOrLoss = WinOrLoss == 'W'
    self.WinID = int(self.WinOrLoss)
    self.LossID = int(not self.WinOrLoss)
    self.PlayerType = PlayerType
    self.TStats = Stats()
    self.OppStats = Stats()
    self.TStatsP = Stats()
    self.OppStatsP = Stats()
  def printDay(self):
    print('Day', self.DayCount)

def aLinetoGame(aline):
  newG = aGame(aline[0], aline[1], aline[2], aline[3], aline[4], aline[5], aline[6],
               aline[7], aline[8], aline[9], aline[10], aline[11])
  for n in range(12, 27):
    aline[n] = int(aline[n])

  newG.TStats.update(aline[12], aline[13], aline[14], aline[15], aline[16], aline[17], 
                     aline[18], aline[19], aline[20], aline[21], aline[22], aline[23], 
                     aline[24], aline[25], aline[26])
  return newG

def readLines(filename):
  with open(filename, newline='') as f:
      reader = csv.reader(f)
      next(reader)
      rawDataL = list(reader)
  #print(rawDataL)
  #'ID', 'GameDay', 'GameTime', 'Home', 'HomeID', 'Visitor', 'VisitorID', 'ScoreVis', 
  #'ScoreHome', 'GameStatus', 'Venue', 'CityState', 'Neutral', 'Division', 'Conference', 'Playoffs'
  #print("RawDataLength ",len(rawDataL))
  data = []
  print(rawDataL[0])
  for oneLine in rawDataL:
    g = aLinetoGame(oneLine)
    data.append(g)
  return data

In [ ]:
from copy import deepcopy
allData = readLines(filename)
print(allData[1])
print(len(allData))

for tg in allData:
  previousGamesList = list(filter(lambda g: (g.DayCount < tg.DayCount) and (g.TeamID == tg.TeamID), allData))
  pMIN = 0
  pPTS = 0
  pFGM = 0
  pFGA = 0
  pFM3 = 0
  pFA3 = 0
  pFTM = 0
  pFTA = 0
  pREB = 0
  pAST = 0
  pSTL = 0
  pBLK = 0
  pOREB = 0
  pTO = 0
  pPF = 0
  pWin = 0
  pLoss = 0
  for p in previousGamesList:
    g = p.TStats
    pMIN += g.MIN
    pPTS += g.PTS
    pFGM += g.FGM
    pFGA += g.FGA
    pFM3 += g.FM3
    pFA3 += g.FA3
    pFTM += g.FTM
    pFTA += g.FTA
    pREB += g.REB
    pAST += g.AST
    pSTL += g.STL
    pBLK += g.BLK
    pOREB += g.OREB
    pTO += g.TO
    pPF += g.PF
    pWin += p.WinID
    pLoss += p.LossID
  gPlayed = len(previousGamesList)
  if gPlayed > 0:
    tg.TStatsP.update(pMIN/gPlayed, pPTS/gPlayed, pFGM/gPlayed, pFGA/gPlayed, pFM3/gPlayed,
                      pFA3/gPlayed, pFTM/gPlayed, pFTA/gPlayed, pREB/gPlayed, pAST/gPlayed,
                      pSTL/gPlayed, pBLK/gPlayed, pOREB/gPlayed, pTO/gPlayed, pPF/gPlayed)
    tg.TStatsP.addWinRate(pWin, pLoss)
  else:
    tg.TStatsP.update(234, 100, 44, 90, 10,
                      30, 20, 25, 45, 25,
                      7, 7, 11, 15, 22)
    tg.TStatsP.addWinRate(1, 1)
    #Random for the first game
allGames = []
for d in allData:
  sameGameList = list(filter(lambda g: g.GameID == d.GameID, allGames))
  if len(sameGameList) > 0:
    for g in sameGameList:
      if g.Home: 
        g.OppStats = deepcopy(d.TStats)
        g.OppStatsP = deepcopy(d.TStatsP)
      else:
        d.OppStats = deepcopy(g.TStats)
        d.OppStatsP = deepcopy(g.TStatsP)
        allGames.remove(g)
        allGames.append(d)
  else:
    allGames.append(d)
allData = allGames
print(len(allData))

['2018-10-16', '985248', '5', 'Boston', '28', 'Philadelphia', 'H', '', '', '', 'W', '', '234', '103', '41', '96', '11', '37', '10', '14', '54', '21', '7', '6', '12', '14', '20']
2580
1290


In [ ]:
def sep2Groups(allD, afterN):#For Ranking
  allDataP = list(filter(lambda g:not g.Playoffs, allD))
  nData = len(allData)
  test_daysAfterN = list(filter(lambda g: g.DayCount >= afterN, allDataP))
  test_daysAfterN = list(map(lambda g: g.DayCount, test_daysAfterN))
  test_days = list(set(test_daysAfterN))

  test_listGroups = []
  train_listGroups = []
  for daysC in test_days:
    print(daysC)
    test_listGroups.append(list(filter(lambda g: g.DayCount >= daysC, allDataP)))
    train_listGroups.append(list(filter(lambda g: g.DayCount < daysC, allDataP)))
  return (test_listGroups, train_listGroups)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def gametoTensor(g : aGame):
  tensor = torch.zeros(1, 1, 78)
  
  tensor[0][0][g.TeamID] = 1
  tensor[0][0][g.OpponentID+37] = 1
  tensor[0][0][77] = g.DayCount/300
  #winloss
  #previous win loss
  
  return tensor

def gamestoTensorTrain(games, test = False):
  g_tensor_list = []
  winRecord = []
  for g in games:
    g_tensor_list.append((gametoTensor(g), g.WinID))
    if g.WinID> 0 :
      winRecord.append(g.TeamID)
    else:
      winRecord.append(g.OpponentID)
  return g_tensor_list, winRecord
def gamestoTensorTest(games, test = False):
  g_tensor_list = []
  for g in games:
    g_tensor_list.append((gametoTensor(g), g.WinID, g.TeamID, g.OpponentID))
  return g_tensor_list

In [ ]:
tempD = allData[0:400]
tIDs = list(set(map(lambda x: x.TeamID, tempD)))
print(len(tIDs))

30


In [ ]:
def allTeamsComb(day):
  rankTestList = []
  tempLine = ['2018-10-16', '985248', '5', 'Boston', '28', 'Philadelphia', 
                'H', '', '', '', 'W', '', '234', '103', '41', '96', '11', '37', 
                '10', '14', '54', '21', '7', '6', '12', '14', '20']
  for a in tIDs:
    for b in tIDs:
      if a != b:
        testG = aLinetoGame(tempLine)
        testG.TeamID = a
        testG.OpponentID = b
        testG.DayCount = day
        rankTestList.append((gametoTensor(testG), a, b))
  return rankTestList

In [ ]:
from collections import Counter
def getRankList(rankRecord):
  #print(len(rankRecord))
  rC = Counter(rankRecord).most_common()

  r = list(map(lambda x: x[0], rC))
  nRank = []
  for a in tIDs:
    if a not in r:
      nRank.append(30)
    else:
      nRank.append(r.index(a)+1)
  return nRank

In [ ]:
allRecord = []
for g in list(filter(lambda g:not g.Playoffs, allData)):
  if g.WinID> 0 :
    allRecord.append(g.TeamID)
  else:
    allRecord.append(g.OpponentID)
print(len(allRecord))
playOffRanking = getRankList(allRecord)
print(playOffRanking)

1208
[7, 21, 17, 6, 9, 26, 1, 27, 28, 2, 13, 15, 29, 18, 19, 10, 12, 25, 30, 14, 23, 4, 3, 8, 20, 5, 22, 11, 16, 24]


In [ ]:
import time
import math
from statistics import mean
import scipy.stats as stats
from scipy.stats import kendalltau
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def runModel(test_listGroups, train_listGroups):
  start = time.time()
  test_tlistGroups = list(map(lambda group: gamestoTensorTest(group, test = True), test_listGroups))
  train_tlistGroups = list(map(lambda group: gamestoTensorTrain(group), train_listGroups))
  totalR_list = []
  print(len(train_tlistGroups), len(test_tlistGroups))
  for t in range(len(train_tlistGroups)):
    print('Model: ', t)
    model = nn.Sequential(nn.Linear(78, 64),
                          nn.Tanh(),
                          nn.Linear(64, 16),
                          nn.Tanh(),
                          nn.Linear(16, 2),
                          nn.LogSoftmax(dim=1))
    #model = model.to(device=device)
    learning_rate = 0.005
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    loss_fn = nn.NLLLoss()#meansquarerror
    n_epochs = 30
    currentRecord = train_tlistGroups[t][1]
    #print(getRankList(currentRecord))
    for epoch in range(n_epochs):
      
      train_loader = torch.utils.data.DataLoader(train_tlistGroups[t][0], batch_size = 1, shuffle = True)
      for gs, labels in train_loader:
        #gs = gs.to(device=device)
        #labels = labels.to(device = device)
        batch_size = gs.shape[0]
        outputs = model(gs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
      if (epoch+1)%10 == 0:
        print(timeSince(start))
        print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

    powerTestList = allTeamsComb(test_listGroups[t][0].DayCount)
    powerRankRecord = []
    for k, a, b in powerTestList:
      outputs = model(k.view(1, -1))
      #print(outputs)
      _, predicted = torch.max(outputs, dim=1)
      #print(predicted)
      if predicted == 1:
        powerRankRecord.append(a)
      else:
        powerRankRecord.append(b)
    powerRankList = getRankList(powerRankRecord)#Power Ranking
    #print(playOffRanking)
    powTau, _ = kendalltau(playOffRanking, powerRankList)
    print(powTau)
    scheduleRecord = []
    for gt, label, tID, oID in test_tlistGroups[t]:
      outputs = model(gt.view(1, -1))
      _, predicted = torch.max(outputs, dim=1)
      if predicted:
        scheduleRecord.append(tID)
      else:
        scheduleRecord.append(oID)
    fakePlayOffRanking = getRankList(currentRecord+scheduleRecord)
    playTau, _ = kendalltau(playOffRanking, fakePlayOffRanking)
    print(playTau)
    totalR_list.append((t, powTau, playTau))
  print(timeSince(start))
  return totalR_list

In [ ]:
def writeInRank(newRecord):
  p1Name='/content/gdrive/My Drive/2021 Fall/Thesis I/Records/Rank2_15.txt'
  p1file=open(p1Name)
  p1file_content = p1file.read()
  p1file=open(p1Name,"w")
  p1file.write(p1file_content)
  for d, pow, p in newRecord:
    p1file.write(str(d)+" "+ str(pow) + " " + str(p) + "\n")
  p1file.write("-\n")
  p1file=open(p1Name)
  p1file_content = p1file.read()
  #print(p1file_content)
  p1file.close()

In [ ]:
trainGs, testGs = sep2Groups(allData, 5)
print(len(trainGs), len(testGs))

5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
175
176
163 163


In [ ]:
for i in range(30):
  print('Run %d' %i)
  writeInRank(runModel(trainGs, testGs))

Run 0
163 163
Model:  0
0m 5s
Epoch: 9, Loss: 0.456481
0m 5s
Epoch: 19, Loss: 0.936383
0m 6s
Epoch: 29, Loss: 0.382236
0.08505747126436781
0.36091954022988504
Model:  1
0m 6s
Epoch: 9, Loss: 0.875965
0m 6s
Epoch: 19, Loss: 0.461787
0m 6s
Epoch: 29, Loss: 0.869464
0.03908045977011494
0.3701149425287356
Model:  2
0m 7s
Epoch: 9, Loss: 0.492843
0m 7s
Epoch: 19, Loss: 0.497836
0m 7s
Epoch: 29, Loss: 0.895822
0.03908045977011494
0.3103448275862069
Model:  3
0m 8s
Epoch: 9, Loss: 0.422913
0m 8s
Epoch: 19, Loss: 0.426285
0m 8s
Epoch: 29, Loss: 0.398269
0.03908045977011494
0.3103448275862069
Model:  4
0m 9s
Epoch: 9, Loss: 0.441591
0m 9s
Epoch: 19, Loss: 0.463133
0m 10s
Epoch: 29, Loss: 0.385698
0.08505747126436781
0.37931034482758624
Model:  5
0m 10s
Epoch: 9, Loss: 0.491418
0m 11s
Epoch: 19, Loss: 0.466661
0m 11s
Epoch: 29, Loss: 0.425480
0.03908045977011494
0.4160919540229885
Model:  6
0m 12s
Epoch: 9, Loss: 0.915100
0m 12s
Epoch: 19, Loss: 0.937694
0m 12s
Epoch: 29, Loss: 1.054449
0.264367

KeyboardInterrupt: ignored